# Sustainable AI for Medical Diagnostics: Reference Implementation

This notebook contains the architectural definitions and functional logic for the models described in the paper **"Sustainable AI for Medical Diagnostics: A Multi-Objective Comparative Study"**.

**Note:** This is a **Minimal Working Example (MWE)** configured for code verification. It uses "toy" hyperparameters (e.g., 1 epoch, reduced simulation timesteps) to demonstrate pipeline functionality without requiring High Performance Computing (HPC) resources. It is **not** intended to reproduce the high-accuracy results (77.3%) or full emissions data presented in the paper.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import snntorch as snn
from snntorch import spikegen
import pennylane as qml
import medmnist
from medmnist import INFO
import numpy as np
import os
from codecarbon import EmissionsTracker

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

## 1. Data Loading (MedMNIST v2)
Loads the OrganAMNIST dataset (Abdominal CT, 11 classes).

In [ ]:
DATASET = 'organamnist'
info = INFO[DATASET]
DataClass = getattr(medmnist, info['python_class'])

# Preprocessing
transform = transforms.Compose([transforms.ToTensor()])

# Load minimal subsets for the Demo
train_ds = DataClass(split='train', transform=transform, download=True)
test_ds = DataClass(split='test', transform=transform, download=True)

# Small batch size for demo purposes
BATCH_SIZE = 16
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

n_classes = len(info['label'])
print(f"Loaded {DATASET} with {n_classes} classes.")

## 2. Model Architectures
This section defines the three core architectures compared in the study:
1. **Classical CNN:** The production baseline.
2. **Neuromorphic SNN:** The energy-efficient proposal.
3. **Hybrid Quantum-Classical:** The experimental comparison.

In [ ]:
# --- 2A. Classical CNN ---
class SmallCNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(), 
            nn.Linear(32 * 7 * 7, 128), nn.ReLU(),
            nn.Linear(128, n_classes)
        )

    def forward(self, x):
        return self.net(x)

# --- 2B. Neuromorphic SNN ---
class SimpleSNN(nn.Module):
    def __init__(self, n_out, beta=0.9):
        super().__init__()
        # No pooling to preserve spatial spikes, matching paper param count approx.
        self.conv = nn.Conv2d(1, 16, 3, padding=1)
        self.lif1 = snn.Leaky(beta=beta)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(16 * 28 * 28, n_out)
        self.lif2 = snn.Leaky(beta=beta)

    def forward(self, x_seq):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        spk2_rec = []

        # Time-loop (SNN dynamics)
        for step in range(x_seq.size(0)):
            x = self.conv(x_seq[step])
            spk1, mem1 = self.lif1(x, mem1)
            x = self.flatten(spk1)
            spk2, mem2 = self.lif2(self.fc(x), mem2)
            spk2_rec.append(spk2)
        
        return torch.stack(spk2_rec, dim=0)

# --- 2C. Quantum Hybrid Head ---
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def vqc_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

class QuantumClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1), nn.ReLU(), nn.MaxPool2d(4),
            nn.Flatten(),
            nn.Linear(8 * 7 * 7, n_qubits) # Reduce to qubit count
        )
        self.weight_shapes = {"weights": (2, n_qubits)}
        self.qlayer = qml.qnn.TorchLayer(vqc_circuit, self.weight_shapes)
        self.fc = nn.Linear(n_qubits, n_classes)

    def forward(self, x):
        x = self.conv(x)
        x = self.qlayer(x)
        return self.fc(x)

## 3. Future Directions: Spectral & Compressed Sensing
These modules implement the Wavelet (DWT), DCT, and Compressive Sensing blocks discussed in the "Future Directions" section of the paper. They are provided here for reproducibility of the extension experiments.

In [ ]:
import pywt
from scipy.fftpack import dct
from sklearn.linear_model import Lasso

def dwt_block(x, wavelet='db2', level=1):
    # Applies Discrete Wavelet Transform
    # x: [B,1,H,W]
    B = x.shape[0]
    out = []
    for i in range(B):
        arr = x[i,0].cpu().numpy()
        coeffs2 = pywt.wavedec2(arr, wavelet, level=level)
        cA = coeffs2[0]
        feat = torch.tensor(cA).float().unsqueeze(0)
        out.append(feat)
    return torch.stack(out, dim=0)

def dct_block(x, keep=0.5):
    # Applies Discrete Cosine Transform with masking
    B,_,H,W = x.shape
    out = np.zeros((B,1,H,W), dtype=np.float32)
    for i in range(B):
        arr = x[i,0].cpu().numpy()
        dct2 = dct(dct(arr.T, type=2, norm='ortho').T, type=2, norm='ortho')
        kH, kW = max(1,int(H*keep)), max(1,int(W*keep))
        mask = np.zeros_like(dct2); mask[:kH,:kW]=1
        out[i,0] = dct2 * mask
    return torch.tensor(out)

print("Experimental preprocessing blocks loaded.")

## 4. Energy Tracking & Demo Execution
This section runs a single-epoch training loop to demonstrate the pipeline is functional. 

**Parameters used for Demo:**
* `Epochs`: 1 (Paper used 50+)
* `SNN Timesteps`: 10 (Paper used 20)
* `Batch Size`: 16 (Paper used 128)

In [ ]:
class EnergyContext:
    def __init__(self, project_name):
        self.tracker = EmissionsTracker(project_name=project_name, measure_power_secs=1, output_dir='.')
    def __enter__(self):
        self.tracker.start()
    def __exit__(self, exc_type, exc_value, traceback):
        self.tracker.stop()

def train_demo(model, loader, model_type='cnn', epochs=1):
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    
    print(f"--- Starting {model_type.upper()} Demo Run ---")
    
    with EnergyContext(project_name=f"demo_{model_type}"):
        for epoch in range(epochs):
            model.train()
            for batch_idx, (data, target) in enumerate(loader):
                data, target = data.to(device), target.to(device).squeeze().long()
                optimizer.zero_grad()
                
                if model_type == 'snn':
                    # Rate encoding for SNN
                    spk_data = spikegen.rate(data, num_steps=10) # Reduced steps for demo
                    spk_out = model(spk_data)
                    # Sum spikes over time for classification
                    output = spk_out.sum(dim=0)
                else:
                    output = model(data)
                
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                
                if batch_idx % 50 == 0:
                    print(f"Epoch {epoch+1} [{batch_idx}/{len(loader)}] Loss: {loss.item():.4f}")
                    break # Stop early for demo

# --- Execute Demos ---
# 1. CNN Demo
cnn = SmallCNN(n_classes=n_classes)
train_demo(cnn, train_loader, model_type='cnn')

# 2. SNN Demo
snn_model = SimpleSNN(n_out=n_classes)
train_demo(snn_model, train_loader, model_type='snn')

print("\nDemo execution complete. Energy logs saved to ./emissions.csv")